In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn.neural_network as nn
import scipy.stats as ss
import statsmodels.api as sm
import sys
import time
from math import sqrt
from sklearn import preprocessing
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from statsmodels.graphics.api import abline_plot

pd.set_option("display.precision",3)
pd.set_option("display.max_rows",10)

In [2]:
datos = pd.read_csv("FOTOSINTESIS.csv", delimiter=";")

In [3]:
datos.head()

,Ano,Cod,Bloque,Altura planta,Area de copa (A) =m2,Diametro tronco (cm),A.F no destructiva m2,P,Asimilacion de CO2,Uso eficiente del agua,...,Conductividad,K,MO,Radiacion,Temperatura,Conductividad electrica,Lluvias acumuladas (mms),Humedad,Numero de dias con lluvia,Total
0,2015,1,1,239.75,4.063,15.700,10.754,8.324,2.36,0.634,...,0.10,0.169,2.084,3.087e+06,26.631,0.018,1457.3,0.242,153,991.987
1,2015,1,2,192.50,5.322,15.600,7.621,8.324,1.73,1.291,...,0.02,0.169,2.084,3.087e+06,26.631,0.018,1457.3,0.242,153,540.582
2,2015,2,2,224.75,7.213,16.350,13.111,8.324,2.72,1.220,...,0.04,0.169,2.084,3.087e+06,26.631,0.018,1457.3,0.242,153,267.533
3,2015,3,2,158.00,2.744,13.025,3.253,8.324,3.20,1.245,...,0.05,0.169,2.084,3.087e+06,26.631,0.018,1457.3,0.242,153,1826.064
4,2015,4,2,196.00,5.926,16.525,8.811,8.324,5.70,1.776,...,0.07,0.169,2.084,3.087e+06,26.631,0.018,1457.3,0.242,153,586.963


In [4]:
datos.describe(include="all")

,Ano,Cod,Bloque,Altura planta,Area de copa (A) =m2,Diametro tronco (cm),A.F no destructiva m2,P,Asimilacion de CO2,Uso eficiente del agua,...,Conductividad,K,MO,Radiacion,Temperatura,Conductividad electrica,Lluvias acumuladas (mms),Humedad,Numero de dias con lluvia,Total
count,428.000,428.000,428.000,428.000,428.000,428.000,428.000,428.000,428.000,428.000,...,428.000,428.000,428.000,4.280e+02,428.000,428.000,428.000,428.000,428.000,390.000
mean,2016.154,30.631,2.007,248.829,9.433,19.854,17.868,9.735,5.360,2.765,...,0.073,0.205,2.365,3.061e+06,25.758,0.024,1276.378,0.229,140.407,2090.800
std,0.762,17.257,0.820,38.097,3.573,3.984,7.517,1.798,3.689,1.375,...,0.057,0.040,0.270,9.400e+05,0.664,0.006,148.459,0.044,46.773,1873.266
min,2015.000,1.000,1.000,144.500,2.090,8.950,2.013,7.998,0.500,0.294,...,0.010,0.146,2.084,1.951e+06,24.870,0.017,1093.900,0.169,83.000,52.780
25%,2016.000,16.000,1.000,223.500,6.691,17.094,12.125,8.324,2.308,1.711,...,0.030,0.169,2.151,2.389e+06,25.284,0.018,1093.900,0.199,83.000,664.110
50%,2016.000,31.000,2.000,252.875,9.366,19.762,17.365,9.221,4.100,2.543,...,0.055,0.221,2.429,2.929e+06,25.686,0.028,1348.100,0.225,153.000,1507.513
75%,2017.000,46.000,3.000,275.542,11.918,22.544,22.888,10.962,8.666,3.655,...,0.102,0.240,2.431,3.232e+06,26.631,0.028,1348.100,0.259,188.000,3078.202
max,2017.000,60.000,3.000,340.500,29.472,30.375,38.558,13.156,13.778,10.631,...,0.301,0.261,2.881,4.756e+06,26.666,0.032,1457.300,0.317,188.000,11619.936


In [ ]:
datos = datos.fillna(datos.mean())

In [ ]:
datos = datos.drop(['Ano','Cod','Bloque'], axis=1)

In [ ]:
cols_to_norm = ['Altura planta','Area de copa (A) =m2', 'Diametro tronco (cm)','A.F no destructiva m2','P','Asimilacion de CO2','Uso eficiente del agua','Transpiracion','Conductividad','K','MO','Radiacion','Temperatura','Conductividad electrica','Lluvias acumuladas (mms)','Humedad',' Numero de dias con lluvia','Total']
datos[cols_to_norm] = datos[cols_to_norm].apply(lambda X: (X - X.min()) / (X.max() - X.min()))

In [ ]:
datos.head()

,Altura planta,Area de copa (A) =m2,Diametro tronco (cm),A.F no destructiva m2,P,Asimilacion de CO2,Uso eficiente del agua,Transpiracion,Conductividad,K,MO,Radiacion,Temperatura,Conductividad electrica,Lluvias acumuladas (mms),Humedad,Numero de dias con lluvia,Total
0,0.486,0.072,0.315,0.239,0.063,0.140,0.033,0.678,0.310,0.2,0.0,0.405,0.98,0.051,1.0,0.491,0.667,0.081
1,0.245,0.118,0.310,0.153,0.063,0.093,0.096,0.205,0.034,0.2,0.0,0.405,0.98,0.051,1.0,0.491,0.667,0.042
2,0.409,0.187,0.345,0.304,0.063,0.167,0.090,0.382,0.103,0.2,0.0,0.405,0.98,0.051,1.0,0.491,0.667,0.019
3,0.069,0.024,0.190,0.034,0.063,0.203,0.092,0.449,0.138,0.2,0.0,0.405,0.98,0.051,1.0,0.491,0.667,0.153
4,0.263,0.140,0.354,0.186,0.063,0.392,0.143,0.577,0.206,0.2,0.0,0.405,0.98,0.051,1.0,0.491,0.667,0.046


In [ ]:
# Función para extraer las columnas(idxSet) de la matriz x. Extraer atributos específicos del conjunto de entrenamiento

def xattrSelect(X,idxSet):
    xOut=[]
    for row in X:
        xOut.append([row[i] for i in idxSet])
    return(xOut)

In [ ]:
total_errors=[]
final_attributes=[]
Ajustes=[]
         
for i in range(10):
    X = datos.values[:,:-1]  #[:,14:18] #[:,3:-1]
    Y = datos.values[:,-1]
    names=datos.drop('Total',axis=1).columns
    X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.20, shuffle=True) #Hacer partición de datos en entrenamiento y prueba
    
    # Aplicación del algoritmo
    
    attributelist=[]                              # Lista vacía donde se irán almacenando los atributos
    index= range(len(X[0]))                       # Número de atributos
    indexSet=set(index)                           # Colección de atributos
    indexSeq=[]
    oosError=[]
    ajus_2=[]
   
    for i in index:
        attSet=set(attributelist)                          # Colección de valores únicos (atributos)
        attTrySet= indexSet-attSet
        attTry=[ii for ii in attTrySet] # Colección de valores por probar (atributos)            attTry=[ii for ii in attTrySet]                    # Crear lista con valores por probar
        errorlist=[]                                       # Crear lista vacía para almacenar errores
        attTemp=[]
        ajus=[]                                            # Crear lista vacía para almacenar R^2

        for iTry in attTry:
            attTemp=[]+attributelist                       # Borra la lista y toma la mejor variable del paso anterior
            attTemp.append(iTry)       # Agrega el índice a la lista para ir tomando las posibles comparaciones de las variables
            xTrainTemp=xattrSelect(X_train,attTemp) # Seleccinar del conjunto de entrenamiento los atributos a probar
            xTestTemp=xattrSelect(X_test,attTemp)   # Seleccinar del conjunto de prueba los atributos a probar
            Xtrain=np.array(xTrainTemp)                    # Convertir las listas en array
            ytrain=np.array(y_train)
            Xtest=np.array(xTestTemp)
            ytest=np.array(y_test)
                           
            mlp = nn.MLPRegressor(solver='adam', hidden_layer_sizes=10, max_iter=100, activation='relu', random_state=1, learning_rate_init=0.025, batch_size='auto', momentum=0.09)
            mlp.fit(Xtrain, ytrain)
                  
            rmsError=np.linalg.norm((ytest-mlp.predict(Xtest)),2)/sqrt(len(ytest)) # raíz cuadrada del error medio
           
            errorlist.append(rmsError)
            attTemp=[]
            Ajuste=r2_score(ytest,mlp.predict(Xtest))
            ajus.append(Ajuste)
        iBest=np.argmin(errorlist)
        rbest=np.argmax(ajus)
        attributelist.append(attTry[iBest])
        oosError.append(errorlist[iBest])
        ajus_2.append(ajus[rbest])
    total_errors.append(oosError)
    Ajustes.append(ajus_2)
    final_attributes.append(attributelist)

In [ ]:
final_attributes

In [ ]:
datos = datos.drop(['Altura planta','Area de copa (A) =m2','Asimilacion de CO2','Uso eficiente del agua','Transpiracion','Conductividad','K','MO','Conductividad electrica','Lluvias acumuladas (mms)','Humedad'], axis=1)

In [ ]:
# Especificar X y Y
X = datos.values[:,:-1]  
Y = datos.values[:,-1]

In [ ]:
# hacer validación cruzada
kf = KFold(n_splits=5,random_state=0, shuffle=True)
kf.get_n_splits(X)

In [ ]:
#CON VALIDACIÓN CRUZADA

start = time.time()

R = []
MSE = []   
MAE = []   
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    mlp = nn.MLPRegressor(solver='adam', hidden_layer_sizes=10, max_iter=100, activation='relu', random_state=1, learning_rate_init=0.025, batch_size='auto', momentum=0.09)
    mlp.fit(X_train, y_train)
    y_pred = mlp.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print("R2: ", r2)
    print("MSE: ", mse)
    print("RMSE: ", np.sqrt(mse))
    print('MAE: ', mae)
    R.append(r2)
    MSE.append(mse)
    MAE.append(mae)
print("meanR2: ",np.array(R).mean())
print("meanMSE: ",np.array(MSE).mean())
print("meanRMSE: ", np.sqrt(MSE).mean())
print('meanMAE: ',np.array(MAE).mean())


end = time.time()
print('Total Time: ', end - start)

In [ ]:
names

In [ ]:
datos.head()

In [ ]:
# Se determinan las variables más frecuentes dentro de las 10 más importantes 

v=[]
for i in range(10):
    for j in range(len(final_attributes)):
        v.append(final_attributes[j][i])

frec=dict()
for i in v:
    if i not in frec:
        frec[i]=1
    else:
        frec[i]+=1

variables=[]

for i in frec.keys():
    if frec[i]>=6:
        variables.append(i)

variables
nameList=[names[i] for i in variables]

In [ ]:
nameList

In [ ]:
# Cambiar las dimensiones de las listas para posteriormente graficarlas
a=np.array(total_errors)
b=[]
for i in range(len(names)):
    for j in a:
        b.append(j[i])
        
b=np.array(b)
b=b.reshape(len(names),10)

In [ ]:
x=range(len(total_errors[0]))
plt.figure()
plt.plot(x, b,":")
plt.plot(x,b.mean(axis=-1),linewidth=2,label="RMSE medio")
plt.title("Atributos a seleccionar")
plt.xlabel("# atributos")
plt.ylabel("RMSE")
plt.axis("tight")
plt.legend()
plt.savefig("RMSE_medio.jpg",dpi=300);

In [ ]:
a=total_errors[0][0]
err=[]
for i in range(len(total_errors[0])):
    err.append(np.abs((a-total_errors[0][i])/a)*100)
    a=total_errors[0][i]
print("La variación del error después de la 10 variables es de: %.2f " %err[11] + "%")

In [ ]:
c=np.array(Ajustes)
d=[]
for i in range(len(names)):
    for j in c:
        d.append(j[i])
        
d=np.array(d)
d=d.reshape(len(names),10)

In [ ]:
# Gráfico de error vs número de atributos
x=range(len(ajus_2))

plt.plot(x,d,"-",linestyle=":")
plt.plot(x,d.mean(axis=-1),linewidth=2,label="Ajuste medio")
plt.xlabel("Número de atributos")
plt.ylabel("Coeficiente de determinación")
plt.legend()
plt.title("Ajuste medio")
plt.xlabel("# atributos")
plt.ylabel("Coeficiente de determinación")
plt.axis("tight")
plt.savefig("R^2_medio.jpg",dpi=300);